In [1]:
# Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
merged_data = pd.read_csv("C:/Users/yunus/Downloads/Formula1-2/merged_data.csv", low_memory=False)

In [3]:
# Merged_Data
# Display the first few rows and summary info
merged_data.head(), merged_data.info(), merged_data.describe(), merged_data.describe(include='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23660 entries, 0 to 23659
Data columns (total 92 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EventName           23660 non-null  object 
 1   Year                23660 non-null  int64  
 2   Time_x              23660 non-null  object 
 3   Driver              23660 non-null  object 
 4   DriverNumber_left   23660 non-null  int64  
 5   LapTime             23510 non-null  object 
 6   LapNumber           23660 non-null  float64
 7   Stint               23660 non-null  float64
 8   PitOutTime          697 non-null    object 
 9   PitInTime           706 non-null    object 
 10  Sector1Time         23199 non-null  object 
 11  Sector2Time         23634 non-null  object 
 12  Sector3Time         23610 non-null  object 
 13  Sector1SessionTime  23147 non-null  object 
 14  Sector2SessionTime  23634 non-null  object 
 15  Sector3SessionTime  23610 non-null  object 
 16  Spee

(            EventName  Year                  Time_x Driver  DriverNumber_left  \
 0  Bahrain Grand Prix  2024  0 days 01:01:37.510000    VER                  1   
 1  Bahrain Grand Prix  2024  0 days 01:03:13.806000    VER                  1   
 2  Bahrain Grand Prix  2024  0 days 01:04:50.559000    VER                  1   
 3  Bahrain Grand Prix  2024  0 days 01:06:27.206000    VER                  1   
 4  Bahrain Grand Prix  2024  0 days 01:08:04.379000    VER                  1   
 
                   LapTime  LapNumber  Stint PitOutTime PitInTime  ...  \
 0  0 days 00:01:37.284000        1.0    1.0        NaN       NaN  ...   
 1  0 days 00:01:36.296000        2.0    1.0        NaN       NaN  ...   
 2  0 days 00:01:36.753000        3.0    1.0        NaN       NaN  ...   
 3  0 days 00:01:36.647000        4.0    1.0        NaN       NaN  ...   
 4  0 days 00:01:37.173000        5.0    1.0        NaN       NaN  ...   
 
                Time_left2 Status_right Points Time_right2 S

In [4]:
# Feature Engineering: Extracting Key Elements
# Number of Stints, Tire Compounds, Laps per Stint, and Average Lap Time per Stint

# Convert LapTime to timedelta for easier time calculations
merged_data['LapTime'] = pd.to_timedelta(merged_data['LapTime'])

# Calculate Laps per Stint, Tire Compounds, and Average Lap Time per Stint

# Group by Driver, Stint, EventName, and Year to calculate for each driver and stint in each event
stint_analysis = merged_data.groupby(['Driver', 'Stint', 'EventName', 'Year']).agg(
    Number_of_Laps=('LapNumber', 'count'),  # Laps per Stint
    Used_Tire_Compound=('Compound', 'first'),  # Tire compound used in this stint
    Avg_Lap_Time=('LapTime', 'mean')  # Average lap time per stint
).reset_index()

print(stint_analysis)

     Driver  Stint              EventName  Year  Number_of_Laps  \
0       ALB    1.0  Australian Grand Prix  2024               6   
1       ALB    1.0    Austrian Grand Prix  2024              12   
2       ALB    1.0     Bahrain Grand Prix  2024              15   
3       ALB    1.0     Belgian Grand Prix  2024               8   
4       ALB    1.0     British Grand Prix  2024              27   
...     ...    ...                    ...   ...             ...   
1080    ZHO    4.0     British Grand Prix  2024              11   
1081    ZHO    4.0    Canadian Grand Prix  2024              17   
1082    ZHO    4.0     Chinese Grand Prix  2024              16   
1083    ZHO    4.0    Japanese Grand Prix  2024               1   
1084    ZHO    5.0     British Grand Prix  2024              13   

     Used_Tire_Compound              Avg_Lap_Time  
0                MEDIUM 0 days 00:01:28.184833333  
1                MEDIUM    0 days 00:01:12.666250  
2                  SOFT    0 days 00:01

In [5]:
stint_analysis.head()

,Driver,Stint,EventName,Year,Number_of_Laps,Used_Tire_Compound,Avg_Lap_Time
0,ALB,1.0,Australian Grand Prix,2024,6,MEDIUM,0 days 00:01:28.184833333
1,ALB,1.0,Austrian Grand Prix,2024,12,MEDIUM,0 days 00:01:12.666250
2,ALB,1.0,Bahrain Grand Prix,2024,15,SOFT,0 days 00:01:39.246800
3,ALB,1.0,Belgian Grand Prix,2024,8,MEDIUM,0 days 00:01:52.141875
4,ALB,1.0,British Grand Prix,2024,27,MEDIUM,0 days 00:01:35.079407407


In [6]:
# Feature Engineering: Calculate Number of Stints

# Calculate the number of stints for each driver in each event (Grand Prix)
stint_count = merged_data.groupby(['Driver', 'EventName', 'Year'])['Stint'].nunique().reset_index()
stint_count.columns = ['Driver', 'EventName', 'Year', 'Number_of_Stints']

print(stint_count)

    Driver                 EventName  Year  Number_of_Stints
0      ALB     Australian Grand Prix  2024                 3
1      ALB       Austrian Grand Prix  2024                 3
2      ALB        Bahrain Grand Prix  2024                 3
3      ALB        Belgian Grand Prix  2024                 3
4      ALB        British Grand Prix  2024                 3
..     ...                       ...   ...               ...
394    ZHO        Mexican Grand Prix  2023                 3
395    ZHO          Miami Grand Prix  2024                 2
396    ZHO         Monaco Grand Prix  2024                 3
397    ZHO  Saudi Arabian Grand Prix  2024                 2
398    ZHO        Spanish Grand Prix  2024                 3

[399 rows x 4 columns]


In [7]:
# Final Data for Model 
# Number_of_Laps, Used_Tire_Compound, Avg_Lap_Time, Number_of_Stints

# Step 3: Merge the stint analysis with the original DataFrame
merged_data1 = pd.merge(merged_data, stint_analysis, on=['Driver', 'Stint', 'EventName', 'Year'], how='left')

# Step 4: Merge the stint count with the original DataFrame
merged_data2 = pd.merge(merged_data1, stint_count, on=['Driver', 'EventName', 'Year'], how='left')

# Now the merged_data contains the engineered features
# Show the updated dataframe with new columns

print(merged_data2)
merged_data2.info()

                EventName  Year                  Time_x Driver  \
0      Bahrain Grand Prix  2024  0 days 01:01:37.510000    VER   
1      Bahrain Grand Prix  2024  0 days 01:03:13.806000    VER   
2      Bahrain Grand Prix  2024  0 days 01:04:50.559000    VER   
3      Bahrain Grand Prix  2024  0 days 01:06:27.206000    VER   
4      Bahrain Grand Prix  2024  0 days 01:08:04.379000    VER   
...                   ...   ...                     ...    ...   
23655  Mexican Grand Prix  2023  0 days 02:59:59.540000    PIA   
23656  Mexican Grand Prix  2023  0 days 03:01:22.902000    PIA   
23657  Mexican Grand Prix  2023  0 days 03:02:46.070000    PIA   
23658  Mexican Grand Prix  2023  0 days 03:04:09.050000    PIA   
23659  Mexican Grand Prix  2023  0 days 03:05:32.471000    PIA   

       DriverNumber_left                LapTime  LapNumber  Stint PitOutTime  \
0                      1 0 days 00:01:37.284000        1.0    1.0        NaN   
1                      1 0 days 00:01:36.296000

In [8]:
# Check final data
merged_data2.head()

,EventName,Year,Time_x,Driver,DriverNumber_left,LapTime,LapNumber,Stint,PitOutTime,PitInTime,...,Status_left3,Message,Days_left3,Time,Status_right3,Days_right3,Number_of_Laps,Used_Tire_Compound,Avg_Lap_Time,Number_of_Stints
0,Bahrain Grand Prix,2024,0 days 01:01:37.510000,VER,1,0 days 00:01:37.284000,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17,SOFT,0 days 00:01:37.162294117,3
1,Bahrain Grand Prix,2024,0 days 01:03:13.806000,VER,1,0 days 00:01:36.296000,2.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17,SOFT,0 days 00:01:37.162294117,3
2,Bahrain Grand Prix,2024,0 days 01:04:50.559000,VER,1,0 days 00:01:36.753000,3.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17,SOFT,0 days 00:01:37.162294117,3
3,Bahrain Grand Prix,2024,0 days 01:06:27.206000,VER,1,0 days 00:01:36.647000,4.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17,SOFT,0 days 00:01:37.162294117,3
4,Bahrain Grand Prix,2024,0 days 01:08:04.379000,VER,1,0 days 00:01:37.173000,5.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17,SOFT,0 days 00:01:37.162294117,3


In [12]:
# Data Preprocessing
# 1. Handling Missing Values
# Drop columns with very high missingness and repeated data due to merge
merged_data2.drop(['Deleted', 'FastF1Generated', 'IsAccurate', 'Message', 'IsPersonalBest', 'Q1', 'Q2', 'Q3','PitOutTime', 'PitInTime', 
                  'DeletedReason', 'HeadshotUrl', 'Time_right2', 'Status_left3', 'Days_left3', 'Time', 'Status_right3', 'Days_right3',
                 'Time_y', 'Date_left', 'Source_left', 'Time_left', 'SessionTime_left', 'Days_left', 'Date_right', 'Status_left', 
                  'Source_right', 'Time_right', 'SessionTime_right', 'Days_right', 'DriverNumber_right', 'BroadcastName', 'DriverId', 
                  'TeamName', 'TeamColor', 'TeamId', 'FirstName', 'LastName', 'FullName', 'CountryCode', 'Time_left2', 'Status_right', 
                   'LapStartDate', 'Days_x', 'Days_y'],
                 axis=1, inplace=True)


In [9]:
# Save the DataFrame to an Excel file
merged_data2.to_excel('output.xlsx', index=False)

In [10]:
# Save the DataFrame to CSV
merged_data2.to_csv('output.csv', index=False)